In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
from pandas.plotting import autocorrelation_plot
import plotting_functions
import visvis as vv
import diagplots2
import pymc3
import matplotlib
import seaborn as sns
sns.reset_orig()

import geovis_notebook_version
from diagplots2 import fieldobs_lookup, display_ground_truth

%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [ ]:
try:
    plt.rc('text', usetex = True)
    plt.rc('font', family = 'serif')
except:
    pass

In [ ]:
author = 'dk' # 'dk' or 'rs

rp_param_title_list = ['Rock density', 'Magnetic susceptibility']
rp_layer_title_list = ['Layer 0', 'Layer 1']

if author == 'dk': # user input - for david's results
    dir_parent = '/Volumes/david_hd/obsidian/output/experiments/'
    dir_output_diagnostics = 'output/plots-diagnostic'
    dir_output_sensors = 'output/plots-sensors'
    fname_samples = 'output0.npz'

    dir_year = '2018'
    dir_month = '11'
    dir_day = '15'
    exp_str = '{}_{}_{}'.format(dir_month, dir_day, dir_year)
    num = '01'

    param_key_list = [
        'layer0rockProperties',
        'layer1rockProperties'
    ]
    param_idx_list = [0, 1]
    # for sensor plot output
    sensor_name_list = ['grav', 'mag'] #'grav' or 'mag'

    data_names_list = [
        'magSensors','magReadings','gravSensors','gravReadings','fieldobsSensors', 'fieldobsReadings'
    ]
    xp = lambda exp_str, num: os.path.join(exp_str, num)
# put together file paths
if author == 'dk':
    fpath_in_list = os.path.join(dir_parent, exp_str, num, fname_samples)
    dir_out_list = os.path.join(dir_parent, exp_str, num, dir_output_diagnostics)
elif author == 'rs':
    fpath_in_list = os.path.join(dir_parent, exp_str + num, fname_samples)
    dir_out_list = os.path.join(dir_parent, exp_str + num, dir_output_diagnostics)

data_names_list_sensorplots = data_names_list
#data_names_list_sensorplots = ['magSensors_subsamp', 'magReadings_subsamp', 'gravSensors', 'gravReadings']
#data_names_list_sensorplots = ['magSensors_rndsub', 'magReadings_rndsub', 'gravSensors_rndsub', 'gravReadings_rndsub']

fpath_samples_list = os.path.join(dir_parent, xp(exp_str, num), fname_samples)

fpath_csv_list = [
    os.path.join(dir_parent, xp(exp_str, num), "{}".format(data_name) + ".csv")
    for data_name in data_names_list_sensorplots
]
print(fpath_csv_list)

In [ ]:
dir_out = '/Users/davidkohn/Desktop'
sensor_name = 'grav' # 'grav' or 'mag'
tick_fontsize = 20
#ticks = np.arange(0, 16000, 2000)
ticks= [2000, 8000, 14000]
tick_labels = ['{:.0f}'.format(tick / 1000) for tick in ticks]

samples = np.load(fpath_in_list)
if sensor_name == 'grav':
    sensor_locations = np.loadtxt(fpath_csv_list[2],delimiter=',')
    sensor_samples = samples['gravReadings']
    actual = np.loadtxt(fpath_csv_list[3])
    cbar_label = 'Anomaly'
    cbar_labelpad = 20
    hist_width = 0.5
    hist_xlab = 'Data - Model (mgal)'

if sensor_name == 'mag':
    sensor_samples = samples['magReadings']
    sensor_locations = np.loadtxt(fpath_csv_list[0],delimiter=',')
    actual = np.loadtxt(fpath_csv_list[1])
    cbar_label = 'Anomaly'
    cbar_labelpad = 0
    hist_width = 100
    hist_xlab = 'Data - Model (nT)'

x, y, z = sensor_locations.T
N, N2 = sensor_samples.shape
actual_deviation = actual - actual.mean()
sensor_samples_subset = sensor_samples[N/2:]
sensor_samples_deviation_mean = sensor_samples_subset.mean(axis=0) - sensor_samples_subset.mean()
sensor_samples_deviation_95 = np.percentile(sensor_samples_subset, 95, axis = 0) - sensor_samples_subset.mean()
sensor_samples_deviation_5 = np.percentile(sensor_samples_subset, 5, axis = 0) - sensor_samples_subset.mean()
resid = actual_deviation - sensor_samples_deviation_mean

details_contour = {
    'dict_name': 'contours',
    'plot_list': [
        (actual_deviation, 'Actual', 'contour'),
        (sensor_samples_deviation_mean, 'Model Mean', 'contour'),
        (sensor_samples_deviation_5, 'Model 5th Percentile', 'contour'),
        (sensor_samples_deviation_95, 'Model 95th Percentile', 'contour'),
    ],
    'subplots_x': 2, 'subplots_y': 2,
    'sharex': True, 'sharey': True,
    'fname_template': '{}-contour.eps',
    'figsize': (10, 10)
}

details_resid = {
    'dict_name': 'resid',
    'plot_list': [
        (sensor_samples_deviation_mean, 'Model Mean', 'contour'),
        (resid, 'Residual Contour', 'contour'),
        (resid, 'Residual Histogram', 'hist'),
    ],
    'subplots_x': 1, 'subplots_y': 3,
    'sharex': False, 'sharey': False,
    'fname_template': '{}-resid.eps',
    'figsize': (15, 5)
}

plot_dict = details_resid
dict_name = plot_dict['dict_name']
plot_list = plot_dict['plot_list']
subplots_x = plot_dict['subplots_x']
subplots_y = plot_dict['subplots_y']
sharex = plot_dict['sharex']
sharey = plot_dict['sharey']
fname_template = plot_dict['fname_template']
figsize = plot_dict['figsize']

fig, axes = plt.subplots(subplots_x, subplots_y, sharey=sharey, sharex=sharex, figsize=figsize)
for (plot_data, axis_name, plot_type), ax in zip(plot_list, axes.flatten()):
    if plot_type == 'contour':
        if dict_name == 'contours':
            c = ax.tricontourf(x, y, plot_data, alpha=0.5, cmap='viridis')
        else:
            c = ax.tricontourf(x, y, actual_deviation, alpha=0.5, cmap='viridis')
        cbar = plt.colorbar(c, ax=ax)
        cbar_ticks = cbar.get_ticks()
        cbar_ticks = [cbar_ticks[0], cbar_ticks[np.floor(len(cbar_ticks)/2)], cbar_ticks[-1]]
        _ = cbar.set_ticks(cbar_ticks)
        _ = cbar.ax.tick_params(labelsize=tick_fontsize)
        #_ = cbar.set_label(cbar_label, fontsize=tick_fontsize, labelpad=cbar_labelpad, rotation=270)
        _ = cbar.ax.set_title(cbar_label, fontsize=10)
        ax.set_title(axis_name, fontsize=tick_fontsize)
        ax.set_xticks(ticks)
        ax.set_xticklabels(tick_labels, fontsize=tick_fontsize)
        ax.set_yticks(ticks)
        ax.set_yticklabels(tick_labels, fontsize=tick_fontsize)
        if dict_name == 'resid':
            c = ax.tricontour(x, y, plot_data, alpha=0.5, colors='k')
            ax.set_xlabel('Eastings (km)', fontsize=tick_fontsize)
            ax.set_ylabel('Northings (km)', fontsize=tick_fontsize)

    elif plot_type == 'hist':
        weights = np.ones_like(plot_data)/float(len(plot_data))
        if sensor_name == 'grav':
            bins = [-3, -2, -1, 0, 1, 2, 3, 4, 5]
            h = ax.hist(
                plot_data, weights = weights, edgecolor = 'black', fill=False, lw=3,
                bins = bins
            )
        elif sensor_name == 'mag':
            h = ax.hist(
                plot_data, weights = weights, edgecolor = 'black', fill=False, lw=3
            )
        elif sensor_name == 'mag':
            bins = []

        #ax.cla()
        labels = pd.rolling_mean(h[1], 2)[1:]
        height = h[0]
        #ax.bar(labels, height=height, edgecolor = 'black', fill=False, width=hist_width, lw=3)
        ax.axvline(np.percentile(plot_data, 2.5), color = 'b')
        ax.axvline(np.percentile(plot_data, 97.5), color = 'b')
        ax.axvline(plot_data.mean(), color = 'r')
        """closest_mean_idx = np.argmin([np.abs(l - plot_data.mean())  for l in labels])
        selected_labels = [labels[0], labels[closest_mean_idx], labels[-1]]
        label_strs = ['{:.1f}'.format(l) for l in selected_labels]
        _ = ax.set_xticks(selected_labels)
        _ = ax.set_xticklabels(label_strs)"""
        _ = plt.ylabel('Probability', fontsize = tick_fontsize)
        _ = plt.xlabel(hist_xlab, fontsize = tick_fontsize)
        _ = plt.xticks(fontsize = tick_fontsize)
        _ = plt.yticks(fontsize = tick_fontsize)
        yt = ax.get_yticks()
        yt = [yt[0], yt[np.floor(len(yt)/2)], yt[-1]]
        ytl = ['{:.0f}\%'.format(yt_i * 100) for yt_i in yt]
        ax.set_yticks(yt)
        ax.set_yticklabels(ytl)
        ax.set_title(axis_name, fontsize=tick_fontsize)

if dict_name == 'contours':
    ax = fig.add_subplot(111, frameon=False)
    plt.tick_params(labelcolor='none', top='off', bottom='off', left='off', right='off', labelbottom=False, labelleft=False)
    ax.set_xlabel('Eastings (km)', labelpad=30, fontsize=tick_fontsize)
    ax.set_ylabel('Northings (km)', labelpad=20, fontsize=tick_fontsize)
    
plt.tight_layout()

fname = fname_template.format(sensor_name)
fpath_fig = os.path.join(dir_out, fname)
plt.savefig(fpath_fig)
plt.clf()

In [ ]:
dir_out = '/Users/davidkohn/Desktop'
sensor_name = 'grav' # 'grav' or 'mag'
tick_fontsize = 20
#ticks = np.arange(0, 16000, 2000)
contour_ticks = [2000, 8000, 14000]
contour_tick_labels = ['{:.0f}'.format(tick / 1000) for tick in contour_ticks]

details_resid = {
    'dict_name': 'resid',
    'plot_list': [
        ('sensor_samples_deviation_mean', 'Model Mean', 'contour', 'grav'),
        ('resid', 'Residual Contour', 'contour', 'grav'),
        ('resid', 'Residual Histogram', 'hist', 'grav'),
        ('sensor_samples_deviation_mean', 'Model Mean', 'contour', 'mag'),
        ('resid', 'Residual Contour', 'contour', 'mag'),
        ('resid', 'Residual Histogram', 'hist', 'mag'),
        ('fieldobs_actual', 'Actual', 'scatter', 'fieldobs'),
        ('fieldobs_predict', 'Predicted', 'scatter', 'fieldobs'),
        ('resid', 'Residual Histogram', 'hist', 'fieldobs'),
    ],
    'subplots_x': 3, 'subplots_y': 3,
    'sharex': False, 'sharey': False,
    'fname_template': 'all-resid.eps',
    'figsize': (15, 15)
}

plot_dict = details_resid
dict_name = plot_dict['dict_name']
plot_list = plot_dict['plot_list']
subplots_x = plot_dict['subplots_x']
subplots_y = plot_dict['subplots_y']
sharex = plot_dict['sharex']
sharey = plot_dict['sharey']
fname_template = plot_dict['fname_template']
figsize = plot_dict['figsize']

samples = np.load(fpath_in_list)
fig, axes = plt.subplots(subplots_x, subplots_y, sharey=sharey, sharex=sharex, figsize=figsize)
for (plot_data_type, axis_name, plot_type, sensor_name), ax in zip(plot_list, axes.flatten()):
    print(sensor_name)
    if sensor_name == 'grav':
        sensor_locations = np.loadtxt(fpath_csv_list[2],delimiter=',')
        sensor_samples = samples['gravReadings']
        actual = np.loadtxt(fpath_csv_list[3])
        cbar_label = 'Anomaly'
        cbar_labelpad = 20
        hist_width = 0.5
        hist_xlab = 'Data - Model (mgal)'
    elif sensor_name == 'mag':
        sensor_samples = samples['magReadings']
        sensor_locations = np.loadtxt(fpath_csv_list[0],delimiter=',')
        actual = np.loadtxt(fpath_csv_list[1])
        cbar_label = 'Anomaly'
        cbar_labelpad = 0
        hist_width = 100
        hist_xlab = 'Data - Model (nT)'
    elif sensor_name == 'fieldobs':
        predicted = samples['fieldReadings']
        predicted_last = predicted[-1,:]
        actual = pd.read_csv(fpath_csv_list[5], names=['val'], comment='#')
        names = ['x', 'y']
        sensor_locs = pd.read_csv(fpath_csv_list[4], names=names, comment='#')
        x = sensor_locs[names[0]].values
        y = sensor_locs[names[1]].values
        actual_labels = np.array(fieldobs_lookup(actual.val))
        predicted_labels = np.array(fieldobs_lookup(predicted_last))
        subset_bool = actual.val.values != -1
        
        resid = actual.val[subset_bool] - predicted_last[subset_bool]
        hist_width = 0.5
        hist_xlab = 'Residual'
    if (sensor_name == 'grav') or (sensor_name == 'mag'):
        x, y, z = sensor_locations.T
        N, N2 = sensor_samples.shape
        actual_deviation = actual - actual.mean()
        sensor_samples_subset = sensor_samples[N/2:]
        sensor_samples_deviation_mean = sensor_samples_subset.mean(axis=0) - sensor_samples_subset.mean()
        resid = actual_deviation - sensor_samples_deviation_mean
    if plot_data_type == 'resid':
        plot_data = resid
    elif plot_data_type == 'sensor_samples_deviation_mean':
        plot_data = sensor_samples_deviation_mean
    elif plot_data_type == 'fieldobs_actual':
        plot_data = actual_labels
    elif plot_data_type == 'fieldobs_predict':
        plot_data = predicted_labels
    
    if plot_type == 'contour':
        ax.set_aspect('equal')
        if dict_name == 'contours':
            #c = ax.tricontourf(x, y, plot_data, 20, alpha=0.5, cmap='viridis', aspect=100)
            c = ax.tricontourf(x, y, plot_data, alpha=0.5, cmap='viridis', aspect=100)
        else:
            #c = ax.tricontourf(x, y, actual_deviation, 20, alpha=0.5, cmap='viridis', aspect=100)
            c = ax.tricontourf(x, y, actual_deviation, alpha=0.5, cmap='viridis', aspect=100)
        #"""
        cbar = plt.colorbar(c, ax=ax)
        cbar_ticks = cbar.get_ticks()
        cbar_ticks = [cbar_ticks[0], cbar_ticks[np.floor(len(cbar_ticks)/2)], cbar_ticks[-1]]
        _ = cbar.set_ticks(cbar_ticks)
        _ = cbar.ax.tick_params(labelsize=tick_fontsize)
        _ = cbar.ax.set_title(cbar_label, fontsize=10)
        #"""
        ax.set_title(axis_name, fontsize=tick_fontsize)
        ax.set_xticks(contour_ticks)
        ax.set_xticklabels(contour_tick_labels, fontsize=tick_fontsize)
        ax.set_yticks(contour_ticks)
        ax.set_yticklabels(contour_tick_labels, fontsize=tick_fontsize)
        if dict_name == 'resid':
            #c = ax.tricontour(x, y, plot_data, c.levels, alpha=0.5, colors='k')
            c = ax.tricontour(x, y, plot_data, alpha=0.5, colors='k')
            ax.set_xlabel('Eastings (km)', fontsize=tick_fontsize)
            ax.set_ylabel('Northings (km)', fontsize=tick_fontsize)
        ax.set_aspect('equal')

    elif plot_type == 'hist':
        weights = np.ones_like(plot_data)/float(len(plot_data))
        if sensor_name == 'grav':
            bins = [-3, -2, -1, 0, 1, 2, 3, 4, 5]
            h = ax.hist(
                plot_data, weights = weights, edgecolor = 'black', fill=False, lw=3,
                bins = bins
            )
            _ = ax.set_xticks(bins)
            _ = ax.set_xticklabels(bins)
        elif sensor_name == 'mag':
            bins = np.arange(-700, 1200, 200)
            #bins = np.arange(-700, 1200, 100)
            h = ax.hist(
                plot_data, weights = weights, edgecolor = 'black', fill=False, lw=3,
                bins = bins
            )
            _ = ax.set_xticks(bins)
            _ = ax.set_xticklabels(bins)
        elif sensor_name == 'fieldobs':
            bins = [-1,0,1, 2]
            h = ax.hist(
                plot_data, bins = bins, width = 1.0, weights = weights,
                edgecolor = 'black', fill=False, lw=3,
            )
            xtick_locations = [-0.5, 0.5, 1.5]
            _ = ax.set_xticks(xtick_locations)
            _ = ax.set_xticklabels(bins, fontsize = tick_fontsize)
            print(h[0])
            print(np.sum(h[0]))
        if (sensor_name == 'grav') or (sensor_name == 'mag'):
            ax.axvline(plot_data.mean(), color = 'r')
            #closest_mean_idx = np.argmin([np.abs(l - plot_data.mean())  for l in labels])
            #selected_labels = [labels[0], labels[closest_mean_idx], labels[-1]]
            #label_strs = ['{:.1f}'.format(l) for l in selected_labels]
            #_ = ax.set_xticks(selected_labels)
            #_ = ax.set_xticklabels(label_strs, fontsize = tick_fontsize)
            ax.axvline(np.percentile(plot_data, 2.5), color = 'b', linestyle='--')
            ax.axvline(np.percentile(plot_data, 97.5), color = 'b', linestyle='--')
            ax.axvline(plot_data.mean(), color = 'r')
        elif (sensor_name == 'fieldobs'):
            #xticks = [-1, 0, 1]
            #xticklabels = ['{:.0f}'.format(l) for l in xticks]
            #_ = ax.set_xticks(xticks)
            #_ = ax.set_xticklabels(xticklabels, fontsize = tick_fontsize)
            pass
        _ = ax.set_ylabel('Probability', fontsize = tick_fontsize)
        _ = ax.set_xlabel(hist_xlab, fontsize = tick_fontsize)
        #_ = plt.yticks(fontsize = tick_fontsize)
        yt = ax.get_yticks()
        yt = [yt[0], yt[np.floor(len(yt)/2)], yt[-1]]
        ytl = ['{:.0f}\%'.format(yt_i * 100) for yt_i in yt]
        ax.set_yticks(yt)
        ax.set_yticklabels(ytl, fontsize = tick_fontsize)
        ax.set_title(axis_name, fontsize=tick_fontsize)
    elif plot_type == 'scatter':
        ax.set_aspect('equal')
        markers = ['o', '*', 's', 'v']
        colors = ['r', 'k', 'b', 'g']
        marker_idx = 0
        for unq in np.unique(plot_data):
            idx = (plot_data == unq)
            _ = ax.scatter(
                x[idx], y[idx], 
                marker=markers[marker_idx],
                label=unq,
                facecolors='none',
                edgecolors=colors[marker_idx],
            )
            marker_idx += 1
        _ = plt.xlabel('Eastings (km)')
        _ = plt.ylabel('Northings (km)')
        yt = ax.get_yticks()[1:-1]
        ytl = ['{}'.format(yt_i/1000) for yt_i in yt]
        ax.set_yticks(yt)
        ax.set_yticklabels(ytl)
        xt = ax.get_xticks()[1:-1]
        xtl = ['{}'.format(xt_i/1000) for xt_i in xt]
        ax.set_xticks(xt)
        ax.set_xticklabels(xtl)
        ax.legend(loc='upper left')
    
plt.tight_layout()

fname = fname_template
fpath_fig = os.path.join(dir_out, fname)
plt.savefig(fpath_fig)
plt.clf()

In [ ]:
actual_deviation = mag_actual - mag_actual.mean()
sensor_samples_subset = mag_samples[N/2:]
sensor_samples_deviation_mean = sensor_samples_subset.mean(axis=0) - sensor_samples_subset.mean()

In [ ]:
fpath = '/Volumes/david_hd/obsidian/output/experiments/11_15_2018/01/output0.npz'
data = np.load(fpath)

In [ ]:
grav_actual = np.loadtxt(fpath_csv_list[3])
grav_actual_anomaly = grav_actual - grav_actual.mean()
grav_samples = data['gravReadings']
N1, N2 = grav_samples.shape
grav_samples = grav_samples[N/2:]
grav_samples_anomaly = grav_samples.mean(axis = 0) - grav_samples.mean()
grav_residual = grav_actual_anomaly - grav_samples_anomaly

mag_samples = data['magReadings']
N1, N2 = mag_samples.shape
mag_samples = mag_samples[N/2:]
mag_actual = np.loadtxt(fpath_csv_list[1])
mag_actual_anomaly = mag_actual - mag_actual.mean()
mag_samples_anomaly = mag_samples.mean(axis = 0) - mag_samples.mean()
mag_residual = mag_actual_anomaly - mag_samples_anomaly

In [ ]:
mag_actual_anomaly

In [ ]:
data_list = [grav_residual, mag_residual]
function_list = [
    np.mean,
    lambda x: 2 * np.std(x),
    lambda x: np.percentile(x, 2.5),
    lambda x: np.percentile(x, 97.5),
]
sensor_name_list = ['Gravity', 'Magnetism']
function_name_list = ['Mean', '2 Sigma', '2.5 Percentile', '97.5 Percentile']

In [ ]:
out_list = [
    [func(data_i) for func in function_list]
    for data_i in data_list
]

In [ ]:
num_data = grav_residual
denom_data = grav_actual
num = 2 * np.std(num_data)
denom = (np.max(denom_data) - np.min(denom_data))
pct_of_range = (num / denom) * 100
print(pct_of_range)
out_list[0].append(pct_of_range)

In [ ]:
num_data = mag_residual
denom_data = mag_actual
num = 2 * np.std(num_data)
denom = (np.max(denom_data) - np.min(denom_data))
pct_of_range = (num / denom) * 100
print(pct_of_range)
out_list[1].append(pct_of_range)

In [ ]:
function_name_list.append('2 Sigma as % of actual range')

In [ ]:
df = pd.DataFrame(out_list, columns = function_name_list, index=sensor_name_list)
df.to_csv('/Users/davidkohn/Desktop/sensor-residual-stats-table.csv')

In [ ]:
df

In [ ]:
# calculate fieldobs posterior probability of success
fo_samples = samples['fieldReadings']
fo_actual = pd.read_csv(fpath_csv_list[5], names=['val'], comment='#')
drop_bool = (fo_actual.values != -1).flatten()
fo_samples_sub = fo_samples[:, drop_bool]
fo_actual_sub = fo_actual[drop_bool].values

p_list = []
N1, N2 = fo_samples_sub.shape
for row_idx in range(N1):
    result = fo_samples_sub[row_idx,:].flatten() == fo_actual_sub.flatten()
    row_p = result.sum()
    p_list.append(row_p)
p_list = np.array(p_list) / N2

In [ ]:
fig = plt.figure(figsize=(5,5))
ax = plt.gca()
plot_data = p_list
weights = np.ones_like(plot_data)/float(len(plot_data))
bins = [0.50, 0.55, 0.60, 0.65, 0.70, 0.75, 0.80, 0.85, 0.90]
h = ax.hist(
    plot_data, weights = weights,
    edgecolor = 'black', fill=False, lw=3,
    bins = bins
)
"""
ax.cla()
labels = pd.rolling_mean(h[1], 2)[1:]
height = h[0]
ax.bar(labels, height=height, edgecolor = 'black', fill=False, width=0.01, lw=3)
"""

#ax.axvline(plot_data.mean(), color = 'r')
closest_mean_idx = np.argmin([np.abs(l - plot_data.mean())  for l in labels])
#selected_labels = [labels[0], labels[closest_mean_idx], labels[-1]]
#selected_labels = [label for idx, label in enumerate(labels) if idx % 2 == 0]
#label_strs = ['{:.2f}'.format(l) for l in selected_labels]
#_ = ax.set_xticks(selected_labels)
#_ = ax.set_xticklabels(label_strs, fontsize = tick_fontsize)
#ax.tick_params(axis='x', labelsize=tick_fontsize)
#ax.set_xticklabels(fontsize=tick_fontsize)
ax.axvline(plot_data.mean(), color = 'r')

yt = ax.get_yticks()
#yt = [yt[0], yt[np.floor(len(yt)/2)], yt[-1]]
ytl = ['{:.0f}\%'.format(yt_i * 100) for yt_i in yt]
ax.set_yticks(yt)
ax.set_yticklabels(ytl, fontsize = tick_fontsize)

xt = bins
xtl = ['{:.0f}\%'.format(xt_i * 100) for xt_i in xt]
ax.set_xticks(xt)
ax.set_xticklabels(xtl, fontsize = 12)

ax.set_title('Field Observation Posterior $\\hat{p}$', fontsize=tick_fontsize)
plt.savefig('/Users/davidkohn/Desktop/fieldobs-p.eps')

In [ ]:
tick_fontsize